In [1]:
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
from sklearn.feature_extraction.text import TfidfVectorizer, TfidfTransformer
from sklearn.decomposition import TruncatedSVD
from sklearn.cluster import KMeans
from mypackage import dir
from auxiliares_doc import listado_palabras

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# Grafico 1

Con los TfidfVectorizer se hizo la nube de 100 palabras

In [2]:
# Environment variables
project = 'belgium'
data = dir.make_dir(project) 
processed = data('processed')
outputs = data('outputs')

In [3]:
# Función para cargar datos
def cargar_datos(table_name: str) -> pd.DataFrame:
    df = pd.read_parquet(processed / f'{table_name}.parquet.gzip')
    print(f'Loaded table: {table_name}')
    return df

def eliminar_palabras(texto, palabras_a_eliminar):
    palabras = texto.split()
    palabras_filtradas = [palabra for palabra in palabras if palabra not in palabras_a_eliminar]
    return ' '.join(palabras_filtradas)

def generar_100_words(df: pd.DataFrame, title: str, save=False):
    """
    Genera y muestra un grafico 100 palabras

    Args:
        df (pd.DataFrame): Dataframe limpio.
        titulo (str): Título para el gráfico.
        save ( ): Indicion para guardar.
    """
    fig = px.scatter(df, x="x", y="y", text="words", 
                     size_max=100, color="group", 
                     labels=labels)

    # Customize text positioning (consider clarity and visual balance)
    fig.update_traces(textposition='top center')  # or 'top center' based on preference

    # Template
    fig.update_layout(template = "plotly_dark")

    # Informative layout with custom title
    fig.update_layout(
        title_text=f'Cloud <br><sup> {title} </sup>',  # Title
        title_x=0.5,  # Centered title
        xaxis_title=' ',  # Add meaningful axis labels
        yaxis_title=' ',
        font=dict(family="Arial", size=12),  # Consistent font style and size
        # plot_bgcolor='white',  # Clean background color
        xaxis_tickformat='.2f',  # Format x-axis ticks for readability (adjust as needed)
        yaxis_tickformat='.2f',  # Format y-axis ticks for readability (adjust as needed)

        xaxis=dict(
            ticks='',
            showticklabels=False
        ),
        yaxis=dict(
            ticks='',
            showticklabels=False
        )

    )

    # Interactive elements (optional)
    fig.update_layout(hovermode='closest')  # Hover over a point for details
    fig.update_traces(opacity=0.8)  # Adjust marker opacity for better visibility

    # Advanced styling (optional)
    fig.update_xaxes(showline=False, linewidth=2, linecolor='gray')  # X-axis formatting
    fig.update_yaxes(showline=False, linewidth=2, linecolor='gray')  # Y-axis formatting

    if save:
        fig.write_html(f'{title}.html'.format('cloud'))

    # Display the enhanced plot
    fig.show()

def build_100_words(df: pd.DataFrame) -> pd.DataFrame:
    """
    Genera un dataset para visualización procesando texto y agrupando palabras clave.
    
    Args:
        df: DataFrame que contiene la columna 'text_noun' con texto preprocesado.
        
    Returns:
        DataFrame con palabras clave, sus coordenadas 2D y grupos de clusterización.
    """
    # Vectorización TF-IDF
    vectorizer = TfidfVectorizer(
        ngram_range=(1, 3),
        max_features=100,
        use_idf=True,
        min_df=4
    )
    tfidf_matrix = vectorizer.fit_transform(df['text_noun'])
    feature_names = [name.replace('_', ' ') for name in vectorizer.get_feature_names_out()]
    
    # Crear DataFrame y filtrar filas con suma cero
    occurrence_df = pd.DataFrame(
        tfidf_matrix.toarray(), 
        columns=vectorizer.get_feature_names_out()
    )
    row_sums = occurrence_df.sum(axis=1)
    occurrence_df = occurrence_df[row_sums != 0]
    
    # Matriz de co-ocurrencia ponderada
    cooccurrence_matrix = np.dot(occurrence_df.T, occurrence_df)
    tfidf_transformer = TfidfTransformer(norm="l2", smooth_idf=False)
    weighted_matrix = tfidf_transformer.fit_transform(cooccurrence_matrix)
    
    # Reducción de dimensionalidad
    reduced_data = TruncatedSVD(
        n_components=2,
        n_iter=7,
        random_state=42
    ).fit_transform(weighted_matrix)
    
    # Determinación óptima de clusters
    optimal_k = _find_optimal_clusters(reduced_data)
    clusters = KMeans(
        n_clusters=optimal_k,
        random_state=42
    ).fit_predict(reduced_data)
    
    # Construcción del DataFrame final
    result_df = pd.DataFrame(reduced_data, columns=['x', 'y'])
    result_df['group'] = clusters
    result_df['group'] = result_df['group'].astype(str)
    result_df['words'] = [word.capitalize() for word in feature_names]
    
    return result_df


def _find_optimal_clusters(data: np.ndarray, min_k: int = 3, max_k: int = 15) -> int:
    """
    Encuentra el número óptimo de clusters usando el método del codo.
    
    Args:
        data: Datos para clusterizar.
        min_k: Mínimo número de clusters a evaluar.
        max_k: Máximo número de clusters a evaluar.
        
    Returns:
        Número óptimo de clusters determinado.
    """
    sse = []
    for k in range(min_k, max_k + 1):
        kmeans = KMeans(n_clusters=k, random_state=42).fit(data)
        sse.append(kmeans.inertia_)
    
    # Encontrar el punto de codo (mayor cambio en la pendiente)
    k_deltas = np.diff(sse)
    return min_k + np.argmax(np.abs(k_deltas)) + 1 


labels={'x':'X-Axis Label',
        'y':'Y-Axis Label',
        'words':'Word',
        'group':'Group'}

In [4]:
df = cargar_datos('df_hotel_transformados')
df['text_noun'] = df['text_noun'].apply(lambda x: eliminar_palabras(x, listado_palabras))

# df['text_noun'] = df['text_noun'].str.replace('hotel hotel hotel hotel hotel', 'hotel')
# df['text_noun'] = df['text_noun'].str.replace('hotel hotel hotel hotel', 'hotel')
# df['text_noun'] = df['text_noun'].str.replace('hotel hotel hotel', 'hotel')
# df['text_noun'] = df['text_noun'].str.replace('hotel hotel', 'hotel')
# df['text_noun'] = df['text_noun'].str.replace(r'(\bhotel\b\s)+hotel', 'hotel', flags=re.IGNORECASE, regex=True)

words = ['hotel', 'agua', 'museo']
for word in words:
    df['text_noun'] = df['text_noun'].str.replace(fr'(\b{word}\b\s)+{word}', word, flags=re.IGNORECASE, regex=True)

df.head()

Loaded table: df_hotel_transformados


,id,label,texto,text,text_noun,longitud,importancia_tfidf
0,1,2,el mejor lugar para comer sushi. excelente lug...,mejor comer sushi excelente comer sushi toda...,sushi martes ronqueo espectaculo,113,3.833645
1,2,2,vista hermosa!. me sorprendió la maravillosa v...,vista hermosa sorprendio maravillosa vista ciu...,sorprendio cable foto,170,4.635010
2,3,0,"desastroso. buenas noches, ante todo explicar...",desastroso buenas noches explicar hotel dos oc...,noche ocasión julio cambio acogedor ambiente e...,502,7.507874
3,4,1,todo bien excepto que se niegan a dar vasos de...,bien excepto niegan dar vasos agua pesar ser r...,vaso agua restaurante preocupacion producto va...,219,3.936999
4,5,2,el mejor zoologico de méxico. el ambiente es m...,mejor zologico mexico ambiente agradable poder...,zologico ambiente poder,102,3.567779


In [5]:
resultado = df[df['text_noun'].str.contains('jama', na=False, regex=True)]
# resultado = df[df['text_noun'].str.contains(r'(?<!aire\s)acondicionado', regex=True, na=False)]
# resultado = resultado.head()
print(resultado.shape)
resultado.head()

(343, 7)


,id,label,texto,text,text_noun,longitud,importancia_tfidf
347,403,2,una visita obligada si usted viaja a monterrey...,visita obligada si usted viaja monterrey nl me...,visita historia grandeza estructura actividad ...,433,6.159495
942,1094,2,interesante lugar. ubicado frente al capitolio...,interesante ubicado frente capitolio si pasar...,restaurante jama imaginaria escalera entra luz...,227,3.893945
1044,1214,1,un desastre. reservamos con vista al mar y nos...,desastre reservamos vista mar dieron habitacio...,desastre mar siquiera equipaje visita pai jama,457,3.096794
1471,1715,0,hotel de 5 estrellas 'peor jamás. fui a este h...,hotel estrellas peor jamas hotel conferencia s...,jama habitación pid noche favor preparate,381,3.266597
1617,1883,2,el mejor corte de beef jamás probado. carne mu...,mejor corte bef jamas probado carne buena jugo...,corte jama carne tierna maderaje,155,5.181752


In [6]:
resultado.shape

(343, 7)

In [7]:
resultado.iloc[0]['texto']

'una visita obligada si usted viaja a monterrey nl mexico. más allá de la historia que este lugar tiene, con la grandeza de sus imponentes estructuras, hornos, estufas, depósitos, hoy convertido en un parque recreativo con muchas actividades, puede alquilar una bicicleta y recorrerlo completamente, puede invertir unas horas formado y hacer el paso en bote, visitar el museo de cera y no perderse, pero nunca jamás, el museo del acero, los sorprenderá! en fin ya sea en plan turista solo, o acompañado o con la familia es un excelente lugar para pasar el día porque hasta tiene un excelente restaurant el lingote.'

In [8]:
df_full = build_100_words(df)
df_full.head()

,x,y,group,words
0,0.152322,-0.048861,2,Acogedor
1,0.296284,-0.139219,1,Acondicionado
2,0.239398,-0.065782,2,Actividad
3,0.301487,0.002676,2,Agua
4,0.211229,-0.140137,1,Amabilidad


In [9]:
generar_100_words(df_full, title='Describe the hotel in 100 words')

C:\Users\Administrator\AppData\Roaming\Python\Python312\site-packages\plotly\express\_core.py:2065: FutureWarning: When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.
  sf: grouped.get_group(s if len(s) > 1 else s[0])


In [10]:
df_positive = df.copy()
df_positive = df_positive[df_positive['label'] == 2]
df_positive = build_100_words(df_positive)
df_positive.head()

,x,y,group,words
0,0.149309,-0.066865,1,Acogedor
1,0.248390,-0.122511,1,Acondicionado
2,0.253039,-0.105374,1,Actividad
3,0.261136,0.024654,0,Agua
4,0.207019,-0.159154,1,Amabilidad


In [11]:
generar_100_words(df_positive, title='Describe positive the hotel in 100 words')

C:\Users\Administrator\AppData\Roaming\Python\Python312\site-packages\plotly\express\_core.py:2065: FutureWarning:

When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.



In [12]:
df_negative = df.copy()
df_negative = df_negative[df_negative['label'] == 0]
df_negative = build_100_words(df_negative)
df_negative.head()

,x,y,group,words
0,0.468593,-0.193707,2,Acondicionado
1,0.125862,0.149178,4,Actitud
2,0.582150,-0.153456,2,Agua
3,0.093541,0.175936,4,Ambiente
4,0.447480,-0.196145,2,Ascensor


In [13]:
generar_100_words(df_negative, title='Describe negative the hotel in 100 words')

C:\Users\Administrator\AppData\Roaming\Python\Python312\site-packages\plotly\express\_core.py:2065: FutureWarning:

When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.

